<a href="https://colab.research.google.com/github/germancruzram/POSPROCESO_PAVIMENTA2/blob/main/post_proceso_pavimentado_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **POSPROCESO A IMAGENES ANALIZADAS EN PAVIMENTADO IADB**
PAVIMENTADO ES UNA APLICACION IA DESARROLLADO POR BID. TODOS LOS DERECHOS DE AUTOR LE PERTENECEN A IADB.

Los códigos python de esta libreta, permite incorporar el nombre de los fotogramas en los que pavimenta2 detectó daños. También incrusta la caja delimitadora y la estacion pkm correspondiente.



**PREPARANDO EL AMBIENTE**

In [ ]:
import os

# Ruta de la carpeta que deseas crear
output_folder = '/content/outputs'

# Verificar si la carpeta ya existe
if not os.path.exists(output_folder):
    # Si no existe, crear la carpeta
    os.makedirs(output_folder)
    print(f'Se ha creado la carpeta: {output_folder}')
else:
    print(f'La carpeta {output_folder} ya existe.')

# Ruta de la carpeta que deseas crear
road_images = '/content/road_images'

# Verificar si la carpeta ya existe
if not os.path.exists(road_images):
    # Si no existe, crear la carpeta
    os.makedirs(road_images)
    print(f'Se ha creado la carpeta: {road_images}')
else:
    print(f'La carpeta {road_images} ya existe.')


Se ha creado la carpeta: /content/outputs
Se ha creado la carpeta: /content/road_images


Antes de continuar cargue los outputs generados por PAVIMENTA2 en la carpeta "outputs".

Cargar las fotografías en "road_images"

**IMPRIMIENDO "data_resulting.csv"**

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Cargar el archivo CSV en un DataFrame
csv_file_path = '/content/outputs/data_resulting.csv'
df = pd.read_csv(csv_file_path)

# Ajustar el formato de los números a 2 decimales
df = df.round(2)

# Definir el estilo de la tabla
html_style = """
<style>
table {
  font-family: Arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
  table-layout: auto; /* Ajustar el ancho de las columnas al contenido */
}

th, td {
  border: 1px solid #dddddd;
  text-align: center; /* Centrar el texto en las celdas */
  padding: 8px;
  white-space: nowrap; /* Evitar que el texto se divida en varias líneas */
  overflow: hidden;
  text-overflow: ellipsis; /* Mostrar puntos suspensivos (...) para el texto que no cabe */
  font-size: 10px; /* Tamaño de letra para las celdas */
  max-width: none; /* Permitir que las celdas se expandan según el contenido */
}

tr:nth-child(even) {
  background-color: #f2f2f2;
}

th {
  background-color: #dddddd;
}
</style>
"""

# Convertir el DataFrame a HTML con el estilo definido y formato de números ajustado
html_table = df.to_html(index=False, float_format='%.2f')

# Combinar el estilo y la tabla en una sola cadena HTML
html_content = f"<html><head>{html_style}</head><body>{html_table}</body></html>"

# Mostrar la tabla HTML
display(HTML(html_content))




latitude,longitude,distances,ind,fotograma,section,classes,ind2,scores,boxes,class_id,area,center,height,width,total_area,perc_area,fail_id_section
12.64,-87.13,6.07,0,0,0,Desenfoque Linea Blanca,0,0.69,"[0.0, 0.28548815846443176, 0.4217282235622406, 0.676917314529419]",Desenfoque Linea Blanca,200099.40,"(615.9395027160645, 199.68831385672092)",399.38,501.03,1212160,0.17,0
12.64,-87.13,5.24,1,1,0,Desenfoque Linea Blanca,1,0.50,"[0.0, 0.28862595558166504, 0.4119471311569214, 0.6884230375289917]",Desenfoque Linea Blanca,199637.01,"(625.3113555908203, 195.05696660280228)",390.11,511.74,1212160,0.16,0
12.64,-87.13,5.24,1,1,0,Otras Fallas,2,0.28,"[0.9008417129516602, 0.25955620408058167, 1.0, 0.28943708539009094]",Otras Fallas,3591.55,"(351.35570526123047, 900.0485510826111)",93.90,38.25,1212160,0.00,15
12.64,-87.13,5.45,2,5,0,Desenfoque Linea Blanca,3,0.75,"[0.8779079914093018, 0.6091248393058777, 1.0, 0.8205727934837341]",Desenfoque Linea Blanca,31293.25,"(915.0064849853516, 889.1894339323044)",115.62,270.65,1212160,0.03,0
12.64,-87.13,5.75,3,14,0,Otras Fallas,4,0.53,"[0.033894132822752, 0.765099287033081, 0.1431542932987213, 0.7950806617736816]",Otras Fallas,3970.76,"(998.5151672363281, 83.83242976851761)",103.47,38.38,1212160,0.00,16
12.64,-87.13,4.89,4,21,1,Desenfoque Linea Blanca,5,0.39,"[0.008190840482711792, 0.3778212070465088, 0.3832918703556061, 0.7402511835098267]",Desenfoque Linea Blanca,164790.55,"(715.5663299560547, 185.3670635819435)",355.22,463.91,1212160,0.14,1
12.64,-87.13,5.03,5,22,1,Desenfoque Linea Blanca,6,0.68,"[0.008672520518302917, 0.3716675639152527, 0.38759511709213257, 0.7595872282981873]",Desenfoque Linea Blanca,178177.25,"(724.0030670166016, 187.6327264085412)",358.84,496.54,1212160,0.15,1
12.64,-87.13,5.37,6,23,1,Desenfoque Linea Blanca,7,0.65,"[0.006155967712402344, 0.3876851797103882, 0.3981863856315613, 0.7724097967147827]",Desenfoque Linea Blanca,182822.52,"(742.4607849121094, 191.45610430836678)",371.25,492.45,1212160,0.15,1
12.64,-87.13,4.22,7,24,1,Desenfoque Linea Blanca,8,0.40,"[0.020588085055351257, 0.3416564464569092, 0.5095723867416382, 0.6547173857688904]",Desenfoque Linea Blanca,185559.74,"(637.6792526245117, 251.0309833958745)",463.07,400.72,1212160,0.15,1
12.64,-87.13,4.64,8,25,1,Desenfoque Linea Blanca,9,0.86,"[0.01950007677078247, 0.37359219789505005, 0.38482922315597534, 0.7468723654747009]",Desenfoque Linea Blanca,165302.41,"(717.0973205566406, 191.44992351531982)",345.97,477.80,1212160,0.14,1


**CREANDO UN MAPA INTERACTIVO**

In [ ]:
!pip install folium
!pip install exifread
!pip install Fraction


In [ ]:
import pandas as pd
import folium

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('/content/outputs/data_resulting.csv')

# Obtener todas las clases únicas presentes en el campo 'classes'
unique_classes = df['classes'].unique()

# Definir una lista de colores para diferenciar cada clase
# Puedes personalizar esta lista según tus preferencias de colores
colors = ['blue', 'green', 'red', 'orange', 'purple', 'gray', 'black', 'pink', 'cyan', 'yellow']

# Crear un diccionario para mapear cada clase única con su color correspondiente
color_dict = {cls: color for cls, color in zip(unique_classes, colors)}

# Crear un mapa centrado en las coordenadas promedio
mapa = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    # Obtener el color correspondiente a la clase
    color = color_dict.get(row['classes'], 'blue')

    # Crear un marcador para cada punto con información relevante
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Clase: {row['classes']}, ID de Clase: {row['class_id']}, Área Perc: {row['perc_area']}",
        icon=folium.Icon(color=color, icon='info-sign')  # Usar un icono con el color correspondiente
    ).add_to(mapa)

# Mostrar el mapa
mapa




**CODIGO PARA ENCONTRAR COINCIDENCIAS FOTOS CON LOS DAÑOS DETECTADOS**

In [ ]:
import os
import pandas as pd
from PIL import Image

# Función para convertir grados, minutos y segundos a grados decimales
def dms_to_decimal(dms, ref):
    degrees = dms[0].numerator / dms[0].denominator
    minutes = dms[1].numerator / dms[1].denominator
    seconds = dms[2].numerator / dms[2].denominator
    flip = -1 if ref in ['S', 'W'] else 1
    return (degrees + (minutes / 60.0) + (seconds / 3600.0)) * flip

# Ruta de los archivos CSV y de las imágenes
csv_file_path = '/content/outputs/data_resulting.csv'
image_folder_path = '/content/road_images'

# Cargar el archivo CSV original
data_df = pd.read_csv(csv_file_path)

# Lista para almacenar los nombres de las imágenes que coinciden
matching_images = []

# Extensiones de archivos de imagen válidas
valid_image_extensions = ['.jpg', '.jpeg', '.png']

# Recorrer cada fila del DataFrame original
for index, row in data_df.iterrows():
    csv_latitude = row['latitude']
    csv_longitude = row['longitude']

    found_match = False

    for filename in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, filename)

        # Verificar si el archivo es de tipo imagen antes de intentar abrirlo
        if os.path.isfile(image_path) and any(image_path.endswith(extension) for extension in valid_image_extensions):
            with Image.open(image_path) as img:
                exif_data = img._getexif()
                if exif_data:
                    # Extraer las coordenadas GPS y sus referencias (N/S y E/W)
                    gps_info = exif_data.get(34853)
                    if gps_info:
                        gps_latitude = gps_info[2]
                        gps_latitude_ref = gps_info[1]
                        gps_longitude = gps_info[4]
                        gps_longitude_ref = gps_info[3]

                        # Convertir a grados decimales
                        img_latitude = dms_to_decimal(gps_latitude, gps_latitude_ref)
                        img_longitude = dms_to_decimal(gps_longitude, gps_longitude_ref)

                        # Comprobar si las coordenadas coinciden con un margen de error
                        if round(img_latitude, 5) == round(csv_latitude, 5) and round(img_longitude, 5) == round(csv_longitude, 5):
                            matching_images.append(filename)
                            found_match = True
                            break

    if not found_match:
        matching_images.append(None)

# Agregar los nombres de las imágenes coincidentes al DataFrame original
data_df['matching_image'] = matching_images

# Identificar el único archivo CSV en la carpeta de imágenes
csv_files = [file for file in os.listdir(image_folder_path) if file.endswith('.csv')]
road_images_csv_path = os.path.join(image_folder_path, csv_files[0])

# Cargar el archivo CSV de la carpeta de imágenes
road_images_df = pd.read_csv(road_images_csv_path)

# Crear un diccionario para mapear los nombres de archivos de imagen a "Punto_kilométrico__km_"
km_point_dict = pd.Series(road_images_df.Punto_kilométrico__km_.values,index=road_images_df.Nombre_de_archivo_de_imagen).to_dict()

# Actualizar el DataFrame original con "Punto_kilométrico__km_" basado en "matching_image"
data_df['PKM'] = data_df['matching_image'].map(km_point_dict)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
data_df.to_csv('/content/outputs/data_resulting_with_images_and_km.csv', index=False)


**CONSTRUCION DE CAJAS DELIMITADORAS**

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image as PILImage
from ast import literal_eval

# Ruta del archivo CSV actualizado con nombres de imágenes coincidentes y cajas delimitadoras
csv_file_path = '/content/outputs/data_resulting_with_images_and_km.csv'

# Ruta de la carpeta que contiene las imágenes
image_folder_path = '/content/road_images/'

# Cargar el archivo CSV
data_df = pd.read_csv(csv_file_path)

# Iterar sobre cada fila del DataFrame
for index, row in data_df.iterrows():
    image_name = row['matching_image']
    image_class = row['classes']
    score = round(row['scores'], 2)  # Redondear el score a dos decimales
    Pkm_item= row['PKM']
    boxes = literal_eval(row['boxes'])  # Convertir la cadena a una lista de Python

    # Mostrar la imagen con la clase, score y nombre de la imagen
    image_path = os.path.join(image_folder_path, image_name)
    with PILImage.open(image_path) as image:
        width, height = image.size

        # Convertir las coordenadas normalizadas a coordenadas absolutas
        x_min = int(boxes[0] * width)
        y_min = int(boxes[1] * height)
        x_max = int(boxes[2] * width)
        y_max = int(boxes[3] * height)

        # Crear una figura y mostrar la imagen
        plt.figure()
        plt.imshow(image)

        # Agregar la caja delimitadora a la imagen
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none'))

        # Mostrar la clase, score y nombre de la imagen en la visualización con un tamaño de fuente más pequeño
    plt.text(10, 20, f'Imagen: {image_name}  Est: {Pkm_item}', color='white', fontsize=8, backgroundcolor='red')
    plt.text(10, 60, f'Clase: {image_class}', color='white', fontsize=8, backgroundcolor='red')
    plt.text(10, 100, f'Score: {score}', color='white', fontsize=8, backgroundcolor='red')

    plt.axis('off')
    plt.show()
    plt.close()  # Cerrar la figura actual para liberar memoria

**GENERAR UN ZIP CON LAS IMAGENES PROCESADAS**

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image as PILImage
from ast import literal_eval
import zipfile

# Ruta del archivo CSV actualizado con nombres de imágenes coincidentes y cajas delimitadoras
csv_file_path = '/content/outputs/data_resulting_with_images_and_km.csv'

# Ruta de la carpeta que contiene las imágenes originales
image_folder_path = '/content/road_images/'

# Ruta de la carpeta para guardar las imágenes procesadas
processed_image_folder_path = '/content/processed_images/'
if not os.path.exists(processed_image_folder_path):
    os.makedirs(processed_image_folder_path)

# Cargar el archivo CSV
data_df = pd.read_csv(csv_file_path)

# Iterar sobre cada fila del DataFrame
for index, row in data_df.iterrows():
    image_name = row['matching_image']
    image_class = row['classes']
    score = round(row['scores'], 2)  # Redondear el score a dos decimales
    Pkm_item = row['PKM']
    boxes = literal_eval(row['boxes'])  # Convertir la cadena a una lista de Python

    # Mostrar la imagen con la clase, score y nombre de la imagen
    image_path = os.path.join(image_folder_path, image_name)
    with PILImage.open(image_path) as image:
        plt.figure()
        plt.imshow(image)
        width, height = image.size

        # Convertir las coordenadas normalizadas a coordenadas absolutas
        x_min = int(boxes[0] * width)
        y_min = int(boxes[1] * height)
        x_max = int(boxes[2] * width)
        y_max = int(boxes[3] * height)

        # Agregar la caja delimitadora a la imagen
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none'))

        # Mostrar la clase, score y nombre de la imagen en la visualización con un tamaño de fuente más pequeño
        plt.text(10, 20, f'Imagen: {image_name}  Est: {Pkm_item}', color='white', fontsize=8, backgroundcolor='red')
        plt.text(10, 60, f'Clase: {image_class}', color='white', fontsize=8, backgroundcolor='red')
        plt.text(10, 100, f'Score: {score}', color='white', fontsize=8, backgroundcolor='red')

        plt.axis('off')

        # Guardar la imagen procesada en el directorio processed_image_folder_path
        processed_image_path = os.path.join(processed_image_folder_path, image_name)
        plt.savefig(processed_image_path)
        plt.close()  # Cerrar la figura actual para liberar memoria

# Comprimir las imágenes procesadas en un archivo ZIP para su descarga
zip_file_path = '/content/processed_images.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for filename in os.listdir(processed_image_folder_path):
        file_path = os.path.join(processed_image_folder_path, filename)
        zipf.write(file_path, os.path.basename(file_path))


print(f'Las imágenes procesadas se han comprimido correctamente en: {zip_file_path}')



Las imágenes procesadas se han comprimido correctamente en: /content/processed_images.zip


In [ ]:
import os
import pandas as pd
from PIL import Image as PILImage, ImageDraw
from ast import literal_eval
import zipfile
import matplotlib.pyplot as plt

# Ruta del archivo CSV actualizado con nombres de imágenes coincidentes y cajas delimitadoras
csv_file_path = '/content/outputs/data_resulting_with_images_and_km.csv'

# Ruta de la carpeta que contiene las imágenes originales
image_folder_path = '/content/road_images/'

# Ruta de la carpeta para guardar las imágenes procesadas
processed_image_folder_path = '/content/processed_images/'
if not os.path.exists(processed_image_folder_path):
    os.makedirs(processed_image_folder_path)

# Cargar el archivo CSV
data_df = pd.read_csv(csv_file_path)

# Comprimir las imágenes procesadas en un archivo ZIP para su descarga
zip_file_path = '/content/processed_images.zip'

# Iterar sobre cada fila del DataFrame
for index, row in data_df.iterrows():
    image_name = row['matching_image']
    image_class = row['classes']
    score = round(row['scores'], 2)  # Redondear el score a dos decimales
    Pkm_item = row['PKM']
    boxes = literal_eval(row['boxes'])  # Convertir la cadena a una lista de Python

    # Mostrar la imagen con la clase, score y nombre de la imagen
    image_path = os.path.join(image_folder_path, image_name)
    with PILImage.open(image_path) as image:
        # Redimensionar la imagen original a 800x600 para la visualización
        image_resized = image.resize((800, 600), PILImage.LANCZOS)

        plt.figure(figsize=(8, 6))
        plt.imshow(image_resized)
        width, height = 800, 600

        # Convertir las coordenadas normalizadas a coordenadas absolutas, ajustadas a 800x600
        x_min = int(boxes[0] * width)
        y_min = int(boxes[1] * height)
        x_max = int(boxes[2] * width)
        y_max = int(boxes[3] * height)

        # Agregar la caja delimitadora a la imagen
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor='r', facecolor='none'))

        # Mostrar la clase, score y nombre de la imagen en la visualización con un tamaño de fuente más pequeño
        plt.text(10, 20, f'Imagen: {image_name}  Est: {Pkm_item}', color='white', fontsize=8, backgroundcolor='red')
        plt.text(10, 60, f'Clase: {image_class}', color='white', fontsize=8, backgroundcolor='red')
        plt.text(10, 100, f'Score: {score}', color='white', fontsize=8, backgroundcolor='red')

        plt.axis('off')

        # Guardar la imagen procesada en el directorio processed_image_folder_path
        processed_image_path = os.path.join(processed_image_folder_path, image_name)
        plt.savefig(processed_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

# Añadir las imágenes procesadas al archivo ZIP y luego eliminarlas
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for filename in os.listdir(processed_image_folder_path):
        file_path = os.path.join(processed_image_folder_path, filename)
        zipf.write(file_path, os.path.basename(file_path))
        os.remove(file_path)  # Eliminar el archivo después de añadirlo al ZIP

print(f'Las imágenes procesadas se han comprimido correctamente en: {zip_file_path}')


Las imágenes procesadas se han comprimido correctamente en: /content/processed_images.zip


**CODIGO PARA LIMPIAR LAS CARPETAS DE DATOS Y REALIZAR OTRO ANALISIS**

In [ ]:
!rm -rf /content/road_images/*
!rm -rf /content/outputs/*
